In [19]:
import torch
import dqc
import time
import pyscf 
from pyscf import gto, dft, cc, scf
import numpy as np 
import numba as nb

### Standard functionals performance and convergence tests

In [37]:
def DQC_HF(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.HF(mol).run()
    ene = qc.energy().detach()  # calculate the energy
    
    return ene

In [38]:
def PySCF_HF(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol)
    ene = hf.kernel(verbose=0)
    return ene

In [39]:
def DQC_KS_PBE(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.KS(mol, xc="gga_c_pbe").run()
    ene = qc.energy().detach()  # calculate the energy
    return ene

In [40]:
def PySCF_KS_PBE(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    ks = dft.RKS(mol)
    ks.xc = "pbe"
    ene = ks.kernel()
    return ene

In [41]:
def PySCF_CCSDT(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol).run()
    cc_ = cc.CCSD(hf)
    ene = cc_.kernel()
    et = cc_.ccsd_t()
    return cc_.e_tot + et

In [42]:
def print_timer(t):
    print(f' took {round(time.time() - t, 3)} s.')

In [43]:
def accuracy_comparison(system, basis):
    t = time.time()
    print("HF, DQC: ", DQC_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("HF, PySCF: ", PySCF_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, DQC: ", DQC_KS_PBE(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, PySCF: ", PySCF_KS_PBE(system, basis))
    print_timer(t)
    t = time.time()
    print("CCSD(T): ", PySCF_CCSDT(system, basis))
    print_timer(t)

Hydrogen at equlibrium point:

In [44]:
accuracy_comparison("H 0 0 0; H 1.4 0 0", "6-31G")

HF, DQC:  tensor(-1.1267, dtype=torch.float64)
 took 0.01 s.
HF, PySCF:  -1.1267427044517944
 took 0.065 s.
PBE, DQC:  tensor(-0.5625, dtype=torch.float64)
 took 0.089 s.
PBE, PySCF:  -1.1619157577671007
 took 0.097 s.
CCSD(T):  -1.1516791660767667
 took 0.091 s.


In [45]:
%timeit DQC_HF("H 0 0 0; H 1.4 0 0", "6-31G")

9.07 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%timeit PySCF_HF("H 0 0 0; H 1.4 0 0", "6-31G")

60.7 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit DQC_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

94.7 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit PySCF_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

105 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Nitrogen at equlibrium point:

In [46]:
accuracy_comparison("N 0 0 0; N 2.07 0 0", "6-31G")

HF, DQC:  tensor(-108.8679, dtype=torch.float64)
 took 0.032 s.
HF, PySCF:  -108.8678750800519
 took 0.069 s.
PBE, DQC:  tensor(-96.5082, dtype=torch.float64)
 took 0.224 s.
PBE, PySCF:  -109.35334923417096
 took 0.174 s.
CCSD(T):  -109.10264680062873
 took 0.223 s.


Ammonia at equlibrium point:

In [51]:
accuracy_comparison("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

HF, DQC:  tensor(-56.1610, dtype=torch.float64)
 took 0.029 s.
HF, PySCF:  -56.16102378396436
 took 0.079 s.
PBE, DQC:  tensor(-49.0140, dtype=torch.float64)
 took 0.401 s.
PBE, PySCF:  -56.451363472681855
 took 0.254 s.
CCSD(T):  -56.29214916490413
 took 0.17 s.


Benzene at equlibrium point:

In [52]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""

In [53]:
accuracy_comparison(benzene_coords, "6-31G")

HF, DQC:  tensor(-216.7411, dtype=torch.float64)
 took 0.914 s.
HF, PySCF:  -216.74175971298365
 took 0.364 s.
PBE, DQC:  tensor(-179.7154, dtype=torch.float64)
 took 5.639 s.
PBE, PySCF:  -218.0955795467209
 took 2.137 s.
CCSD(T):  -217.07797043623327
 took 5.581 s.


Naphtalene at equlibrium point:

In [54]:
naphtalene_coords = """H       -1.057571674      0.000000000     -4.361571149;
                    C       -1.057571674      0.487597890     -3.390034963;
                    C       -1.057571674      1.860916946     -3.299493774;
                    C       -1.057571674      2.493399263     -2.031832408;
                    C       -1.057571674      1.739878824     -0.880133713;
                    C       -1.057571674      0.320033769     -0.937967730;
                    C       -1.057571674     -0.320033769     -2.220831877;
                    C       -1.057571674     -1.739878824     -2.278665893;
                    C       -1.057571674     -2.493399263     -1.126967199;
                    C       -1.057571674     -1.860916946      0.140694168;
                    C       -1.057571674     -0.487597890      0.231235357;
                    H       -1.057571674      2.466184811     -4.201283252;
                    H       -1.057571674      3.577893185     -1.973126484;
                    H       -1.057571674      2.222891720      0.093690092;
                    H       -1.057571674      0.000000000      1.202771542;
                    H       -1.057571674     -2.466184811      1.042483645;
                    H       -1.057571674     -3.577893185     -1.185673122;
                    H       -1.057571674     -2.222891720     -3.252489699"""

In [55]:
accuracy_comparison(naphtalene_coords, "6-31G")

HF, DQC:  tensor(-359.6521, dtype=torch.float64)
 took 5.846 s.
HF, PySCF:  -359.66052906142863
 took 2.0 s.
PBE, DQC:  tensor(-298.3350, dtype=torch.float64)
 took 29.993 s.
PBE, PySCF:  -361.9417376066551
 took 14.763 s.
CCSD(T):  -360.2427068941059
 took 75.314 s.


Anthracene at equlibrium point:

In [57]:
anthracene_coords = """C        1.044178831     -0.528233237     -4.684523771;
C        1.044178831     -0.552713285     -3.254782652;
C        1.044178831     -1.749044624     -2.526815306;
C        1.044178831      0.704131514     -2.541488804;
C        1.044178831      0.692578077     -1.141127549;
C        1.044178831     -0.503753261     -0.413160203;
C        1.044178831     -1.760598060     -1.126454051;
C        1.044178831      0.658108897     -5.368920348;
C        1.044178831      1.919058175     -3.295618865;
H        1.044178831      1.638563069     -0.604255156;
C        1.044178831     -0.528233309      1.016580916;
C        1.044178831     -2.975524721     -0.372323990;
C        1.044178831     -2.954793300      0.997119891;
C        1.044178831     -1.714574573      1.700977987;
H        1.044178831      0.418178836      1.551045394;
H        1.044178831     -3.919661467     -0.910797789;
H        1.044178831     -3.885538136      1.556731594;
H        1.044178831     -1.717772077      2.786998206;
C        1.044178831      1.898326754     -4.665062745;
H        1.044178831     -1.474645383     -5.218988249;
H        1.044178831     -2.695029615     -3.063687699;
H        1.044178831      0.661305531     -6.454941061;
H        1.044178831      2.863194921     -2.757145066;
H        1.044178831      2.829071589     -5.224674449"""

In [58]:
accuracy_comparison(anthracene_coords, "6-31G")

HF, DQC:  tensor(-502.5513, dtype=torch.float64)
 took 20.153 s.
HF, PySCF:  -502.5894979290165
 took 6.88 s.


/home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=1.182e-01, |f|=3.977e-01 at iter 28
  warnings.warn(ConvergenceWarning(msg))


PBE, DQC:  tensor(-416.9599, dtype=torch.float64)
 took 124.682 s.
PBE, PySCF:  -505.8042731691444
 took 35.914 s.
CCSD(T):  -503.42435540834987
 took 920.245 s.


### Custom functionals

Trying to set up HEG LDA functional:

In [60]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

In [62]:
from math import pi

factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0
conversion_factor = (1.8897259885789)**(3 * power - 3)

In [63]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) 
                          + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
a = torch.nn.Parameter(torch.tensor(factor, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(power, dtype=torch.double))
myxc = MyLDAX(a, p)

In [64]:
def dqc_compute(system, basis="6-31G", xc=myxc):
    mol = dqc.Mol(moldesc=system, basis=basis)
    qc = dqc.KS(mol, xc=xc).run()
    ene = qc.energy()
    return ene

In [88]:
en = dqc_compute(water)
en

tensor(-75.1547, dtype=torch.float64, grad_fn=<AddBackward0>)

In [66]:
def my_eval_xc(a,p):    
    def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, verbose=None):
        exc = a * rho**p
        vrho = a * p * rho**(p-1)
        vgamma = None
        vlapl = None
        vtau = None
        vxc = (vrho, vgamma, vlapl, vtau)
        fxc = None  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    return eval_xc

In [71]:
@nb.njit
def exc_jit(rho, a, p):
    return a * rho**p

@nb.njit
def vrho_jit(rho, a, p):
    return a * p * rho**(p-1)

@nb.njit
def eval_xc_jit(xc_code, rho, spin=0, relativity=0, deriv=1, verbose=None):
    a = factor/conversion_factor
    p = power
    exc = exc_jit(rho, a, p)
    vrho = vrho_jit(rho, a, p)
    vgamma = None
    vlapl = None
    vtau = None
    vxc = (vrho, vgamma, vlapl, vtau)
    fxc = None  # 2nd order functional derivative
    kxc = None  # 3rd order functional derivative
    return exc, vxc, fxc, kxc

In [68]:
def scf_compute(system, basis="6-31G", xc = my_eval_xc(factor/conversion_factor,power), verbose=0):
    scf_mol = pyscf.gto.M(atom = system, basis=basis, unit="Bohr")
    mf = pyscf.dft.RKS(scf_mol)
    mf = mf.define_xc_(xc, 'LDA')
    mf.verbose = verbose
    return mf.kernel()

In [86]:
scf_compute(water)

-194.44377477665938

In [87]:
scf_compute(water, xc = eval_xc_jit)

-194.44377477667507

In [89]:
dqc_compute(benzene_coords).detach()

tensor(-213.4143, dtype=torch.float64)

In [91]:
scf_compute(benzene_coords)

-433.106948550808

In [94]:
dqc_compute(naphtalene_coords).detach()

tensor(-354.2951, dtype=torch.float64)

In [95]:
scf_compute(naphtalene_coords)

-722.293818100246